<a href="https://colab.research.google.com/github/nicoussj/MachineLearning/blob/main/SoportVectorMachine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Imports



In [13]:
# Importación de librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm
import warnings
from google.colab import files

# Configuraciones para gráficos
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)

# Ignorar advertencias
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix


In [2]:
# Subir archivo
uploaded = files.upload()
for fn in uploaded.keys():
    name = fn

# Leer CSV
df = pd.read_csv(name, sep=';', encoding='latin1')
df.columns = df.columns.str.replace(';', '')

# Limpiar columnas no necesarias
df = df.drop(columns=['Unnamed: 0', 'AbnormalMatch'], errors='ignore')

# Corregir valores erróneos manuales si existen
df['RoundWinner'] = df['RoundWinner'].replace('False4', 'False')
df['MatchWinner'] = df['MatchWinner'].fillna('False')

# Convertir a binario
df['RoundWinner'] = df['RoundWinner'].astype(str).map({'True': 1, 'False': 0})
df['MatchWinner'] = df['MatchWinner'].astype(str).map({'True': 1, 'False': 0})
df['Survived'] = df['Survived'].astype(str).map({'True': 1, 'False': 0})

# Convertir columnas numéricas que vienen con formato incorrecto
features = ['TimeAlive', 'TravelledDistance', 'RoundKills', 'MatchKills', 'RoundAssists', 'MatchAssists', 'RoundHeadshots']

for col in features:
    df[col] = df[col].astype(str).str.replace('.', '', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Eliminar filas con datos inválidos
df = df.dropna(subset=features + ['RoundWinner'])

Saving Anexo ET_demo_round_traces_2022.csv to Anexo ET_demo_round_traces_2022.csv


In [3]:
# -------------------------------
# 1. Preparar los datos
# -------------------------------

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Definir variable objetivo binaria y características
target_clf = 'RoundWinner'
features_clf = ['MatchHeadshots', 'RoundStartingEquipmentValue', 'TeamStartingEquipmentValue', 'PrimaryAssaultRifle']

# Crear dataset y limpiar
df_model_clf = df[features_clf + [target_clf]].copy()
df_model_clf['PrimaryAssaultRifle'] = pd.to_numeric(df_model_clf['PrimaryAssaultRifle'], errors='coerce')
df_model_clf = df_model_clf.dropna()

# Separar X e y
X = df_model_clf[features_clf]
y = df_model_clf[target_clf]

# Escalar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

In [5]:
# -------------------------------
# 2. Entrenar el modelo SVC
# -------------------------------

# Modelo SVM para clasificación (kernel RBF)
svc_model = SVC(kernel='rbf', C=1, probability=True)
svc_model.fit(X_train, y_train)

SVC(C=1, probability=True)

In [10]:
# -------------------------------
# 3. Entrenar y predecir (clasificación)
# -------------------------------

# Entrenar el modelo SVM de clasificación
svc_model = SVC(kernel='rbf', C=1, probability=True)
svc_model.fit(X_train, y_train)

# Predecir clases
y_pred = svc_model.predict(X_test)

In [14]:

# Evaluación para clasificación
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"📊 Evaluación del modelo SVC:")
print(f"Accuracy:  {acc:.2f}")
print(f"Precision: {prec:.2f}")
print(f"Recall:    {rec:.2f}")
print(f"F1 Score:  {f1:.2f}")

# Extra: Matriz de confusión y reporte completo
print("\nMatriz de confusión:")
print(confusion_matrix(y_test, y_pred))

print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

📊 Evaluación del modelo SVC:
Accuracy:  0.63
Precision: 0.71
Recall:    0.46
F1 Score:  0.56

Matriz de confusión:
[[9547 2279]
 [6389 5532]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.60      0.81      0.69     11826
           1       0.71      0.46      0.56     11921

    accuracy                           0.63     23747
   macro avg       0.65      0.64      0.62     23747
weighted avg       0.65      0.63      0.62     23747

